In [1]:
import json
import pandas as pd
import numpy as np
from emoji import core
import re
import ast
from matplotlib import pyplot as plt

In [2]:
playlists = pd.read_json('../0_data/playlists.json', typ = 'frame', encoding='utf-8')
song_meta = pd.read_json('../0_data/song_meta.json', typ = 'frame', encoding='utf-8') 
genre_gn_all = pd.read_json('../0_data/genre_gn_all.json', typ = 'frame', encoding='utf-8', orient='index')

### 0. genre

In [3]:
# 컬럼명 지정
genre_gn_all = genre_gn_all.reset_index().rename(columns={'index':'gnr_code', 0:'gnr_name'})
genre_gn_all.head()

# 대분류 장르코드들만 뽑기
gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:]=='00']

# 소분류 장르코드만 뽑기
dtl_gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:] != '00']
dtl_gnr_code.rename(columns = {'gnr_code' : 'dtl_gnr_code', 'gnr_name' : 'dtl_gnr_name'}, inplace = True)

# 대분류 장르코드와 소분류 장르코드 각각의 앞자리 네자리 공통코드 추출
gnr_code = gnr_code.assign(join_code = gnr_code['gnr_code'].str[0:4])
dtl_gnr_code = dtl_gnr_code.assign(join_code = dtl_gnr_code['dtl_gnr_code'].str[0:4])

# join_code(네자리 공통코드) 기준으로 Merge
gnr_code_tree = pd.merge(gnr_code, dtl_gnr_code, how = 'left', on = 'join_code')

gnr_code_tree[['gnr_code', 'gnr_name', 'dtl_gnr_code', 'dtl_gnr_name']]
gnr_code_tree

gnr_big = pd.DataFrame(gnr_code.iloc[:, :2])

C:\Users\cg063\AppData\Local\Temp\ipykernel_12508\3749102257.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtl_gnr_code.rename(columns = {'gnr_code' : 'dtl_gnr_code', 'gnr_name' : 'dtl_gnr_name'}, inplace = True)


In [4]:
# song_meta 저장
gnr_big.to_json('../0_data/genre_gn_all_filter.json', orient='records')

### 1. song_meta

> 0. album name이 비어 있는 노래 삭제

In [5]:
song_meta[song_meta['album_name'].isnull()] # album_name 비어 있는 경우 확인

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
143209,"[GN0501, GN0601, GN0503, GN0606, GN0509]",20150522,None,2318271,[726168],사랑바보,"[GN0500, GN0600]",[민은서밴드],143209
291631,"[GN0105, GN0101]",20101108,None,1065178,[28772],빗속에서,[GN0100],[존박],291631
431110,"[GN0509, GN0601, GN0503, GN0606, GN0501]",20200208,None,10385717,[2203064],화면,"[GN0500, GN0600]",[Oowl Hannal (우울 한날)],431110
552088,"[GN0105, GN0601]",20101108,None,1065178,[28263],본능적으로 (Feat. Swings),"[GN0600, GN0100]",[강승윤],552088


In [6]:
song_meta.dropna(subset=['album_name'], inplace=True) # 삭제
song_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707985 entries, 0 to 707988
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   song_gn_dtl_gnr_basket  707985 non-null  object
 1   issue_date              707985 non-null  int64 
 2   album_name              707985 non-null  object
 3   album_id                707985 non-null  int64 
 4   artist_id_basket        707985 non-null  object
 5   song_name               707985 non-null  object
 6   song_gn_gnr_basket      707985 non-null  object
 7   artist_name_basket      707985 non-null  object
 8   id                      707985 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 54.0+ MB


> 1. 날짜 오류 수정

In [7]:
# 날짜 오류있는 행 확인
song_meta[song_meta['issue_date'] > 20220000] # index : 141185, 448286

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
141185,[GN1801],20220113,고요한 밤바다 구경하기 [여수 바다],10403230,[2737142],잔잔한 바람이 불어와 (There is a gentle breeze),[GN1800],[무드홀릭 (Moodholic)],141185
448286,[GN1801],20220113,고요한 밤바다 구경하기 [여수 바다],10403230,[2737142],여수 바다 (Yeosu sea),[GN1800],[무드홀릭 (Moodholic)],448286


In [8]:
indices_to_update = [141185, 448286]
new_value = 20200113 # 알맞는 발매날짜
# 특정 행의 'issue_date' 값을 변경
for idx in indices_to_update:
    song_meta.at[idx, 'issue_date'] = new_value

In [9]:
# 알맞게 변경되었는지 확인
song_meta.iloc[[141185, 448286], :]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
141185,[GN1801],20200113,고요한 밤바다 구경하기 [여수 바다],10403230,[2737142],잔잔한 바람이 불어와 (There is a gentle breeze),[GN1800],[무드홀릭 (Moodholic)],141185
448289,"[GN1003, GN1013, GN1001]",20060103,First Impressions Of Earth,312378,[101138],You Only Live Once,[GN1000],[The Strokes],448289


> 2.song_gn_dtl_gnr_basket가 비어있는 경우와 대분류 장르 GN9000 삭제

In [10]:
song_meta['song_gn_dtl_gnr_basket_cnt'] = song_meta['song_gn_dtl_gnr_basket'].apply(len)
song_meta[song_meta['song_gn_dtl_gnr_basket_cnt']==0]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,song_gn_dtl_gnr_basket_cnt
21,[],20200331,WHY,10410508,[417985],WHY,[],[4minute],21,0
262,[],20150318,노년의 건강을 지켜주는 음악 처방전 (부제 : 휴식 때 들으면 좋은 음악친구),2309524,[726909],Bio Sound `Breathing & Big Bell`,[GN9000],[차병원],262,0
399,[],20200106,뽀로로 겨울 동요,10372098,[1703695],초코 초코 핫초코,[],[아이코닉스 (ICONIX)],399,0
454,[],20191108,MAGDALENE,10349014,[788767],thousand eyes,[GN1100],[FKA Twigs],454,0
803,[],20040901,Tropical Lullaby,153435,[848353],Kalua Lullaby,[],[The Moonlighters],803,0
...,...,...,...,...,...,...,...,...,...,...
706502,[],20110831,엄마와 아가와의 행복한 대화 태교음악 뉴에이지,2009854,[561734],Love Love Love,[GN9000],[해피타임뮤직],706502,0
706716,[],20200109,Have It All,10373946,[2762722],Have It All,[GN9000],[Renegade],706716,0
707201,[],20200115,모두의 MR반주 123,10376105,[770787],Into the Unknown (겨울왕국 2 OST) (Melody MR),[GN9000],[모두의MR],707201,0
707759,[],20200225,낭만닥터 김사부 2 OST,10393897,"[27619, 726783]",Emergency,[],"[전창엽, 마마고릴라]",707759,0


In [11]:
# 소분류 장르가 없는 song id 담아두기
song_gnr_empty = list(song_meta[song_meta['song_gn_dtl_gnr_basket_cnt'] == 0]['id'].values) # 4037개
# 장르정보가 GN9000인 song id 담아두기
song_gnr_GN9000 = list(song_meta[song_meta['song_gn_gnr_basket'].apply(lambda x: 'GN9000' in x)]['id'].values) # 1834개

# 삭제될 song id list
to_del_song_id = set(song_gnr_empty + song_gnr_GN9000)
len(to_del_song_id) # 4040개

4040

In [12]:
song_meta = song_meta[song_meta['song_gn_dtl_gnr_basket_cnt'] != 0] # 소분류 장르가 없는 song 4037개 삭제
song_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 703948 entries, 0 to 707988
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   song_gn_dtl_gnr_basket      703948 non-null  object
 1   issue_date                  703948 non-null  int64 
 2   album_name                  703948 non-null  object
 3   album_id                    703948 non-null  int64 
 4   artist_id_basket            703948 non-null  object
 5   song_name                   703948 non-null  object
 6   song_gn_gnr_basket          703948 non-null  object
 7   artist_name_basket          703948 non-null  object
 8   id                          703948 non-null  int64 
 9   song_gn_dtl_gnr_basket_cnt  703948 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 59.1+ MB


In [13]:
song_meta = song_meta[~song_meta['song_gn_gnr_basket'].apply(lambda x: 'GN9000' in x)] # GN9000가 있는 행 3개삭제 (GN9000이 대분류 장르인 경우, 대부분 소분류 정보가 없어서 여기서는 3개만 삭제됨)
song_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 703945 entries, 0 to 707988
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   song_gn_dtl_gnr_basket      703945 non-null  object
 1   issue_date                  703945 non-null  int64 
 2   album_name                  703945 non-null  object
 3   album_id                    703945 non-null  int64 
 4   artist_id_basket            703945 non-null  object
 5   song_name                   703945 non-null  object
 6   song_gn_gnr_basket          703945 non-null  object
 7   artist_name_basket          703945 non-null  object
 8   id                          703945 non-null  int64 
 9   song_gn_dtl_gnr_basket_cnt  703945 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 59.1+ MB


In [14]:
# 장르 정보가 없는 행이 없는지 확인
song_meta[song_meta['song_gn_dtl_gnr_basket_cnt']==0]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,song_gn_dtl_gnr_basket_cnt


In [15]:
# GN9000가 삭제되었는지 확인
song_meta[song_meta['song_gn_gnr_basket'].apply(lambda x: 'GN9000' in x)]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,song_gn_dtl_gnr_basket_cnt


> 3. 대분류 장르명과 song_meta merge

In [16]:
song_meta_ex = song_meta.explode('song_gn_gnr_basket') # 대분류 장르 한줄에 하나씩 넣기
song_meta_ex.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,song_gn_dtl_gnr_basket_cnt
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,GN0900,[Various Artists],0,1
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",GN1600,[Murray Perahia],1,2
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),GN0900,[Peter Gabriel],2,1
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,GN1100,[Matoma],3,2
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,GN1800,[Jude Law],4,2


In [17]:
# 대분류 장르와 song_meta merge
song_meta_merge = pd.merge(song_meta_ex, gnr_big, how='left', left_on='song_gn_gnr_basket', right_on='gnr_code')
song_meta_merge.drop('gnr_code', axis=1, inplace=True)
song_meta_merge.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,song_gn_dtl_gnr_basket_cnt,gnr_name
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,GN0900,[Various Artists],0,1,POP
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",GN1600,[Murray Perahia],1,2,클래식
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),GN0900,[Peter Gabriel],2,1,POP
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,GN1100,[Matoma],3,2,일렉트로니카
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,GN1800,[Jude Law],4,2,뉴에이지


In [18]:
song_meta_gnr_code = song_meta_merge.groupby('id')['song_gn_gnr_basket'].apply(list).reset_index() # 하나의 song에 대해 하나의 대분류 장르코드 list로 만들기
song_meta_gnr_name = song_meta_merge.groupby('id')['gnr_name'].apply(list).reset_index() # 하나의 song에 대해 하나의 대분류 장르명 list로 만들기

In [19]:
song_meta_f = pd.merge(song_meta_merge, song_meta_gnr_code, how='left', on='id') # 장르코드 list와 merge
song_meta_f = pd.merge(song_meta_f, song_meta_gnr_name, how='left', on='id') # 장르명 list와 merge

In [20]:
# 필요한 컬럼만 남기기
song_meta_f = song_meta_f[['song_gn_dtl_gnr_basket', 'issue_date', 'album_name', 'album_id', 'artist_id_basket', 'song_name', 'song_gn_gnr_basket_y', 'gnr_name_y', 'artist_name_basket', 'id']]
# 컬럼명 수정
song_meta_f.rename(columns={'song_gn_gnr_basket_y':'song_gn_gnr_basket', 'gnr_name_y':'gnr_name'}, inplace=True)

song_meta_f.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,gnr_name,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[POP],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[클래식],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[POP],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[일렉트로니카],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[뉴에이지],[Jude Law],4


In [21]:
song_meta_cleaned = song_meta_f.drop_duplicates(subset='id', keep='first') # id가 중복인 행에 대해 하나의 id행만 남기기

In [22]:
song_meta_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 703945 entries, 0 to 799765
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   song_gn_dtl_gnr_basket  703945 non-null  object
 1   issue_date              703945 non-null  int64 
 2   album_name              703945 non-null  object
 3   album_id                703945 non-null  int64 
 4   artist_id_basket        703945 non-null  object
 5   song_name               703945 non-null  object
 6   song_gn_gnr_basket      703945 non-null  object
 7   gnr_name                703945 non-null  object
 8   artist_name_basket      703945 non-null  object
 9   id                      703945 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 59.1+ MB


In [23]:
# song_meta 저장
song_meta_cleaned.to_json('../0_data/orgi_song_meta.json', orient='records')

> 4. song_meta DB용으로 만들기

In [24]:
song_meta_db = song_meta_cleaned[['id', 'song_name', 'artist_name_basket', 'gnr_name']]
song_meta_db.rename(columns={'id':'song_id', 'artist_name_basket':'artist_name_lst', 'gnr_name':'song_gnr_lst'}, inplace=True)

C:\Users\cg063\AppData\Local\Temp\ipykernel_12508\593262896.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_meta_db.rename(columns={'id':'song_id', 'artist_name_basket':'artist_name_lst', 'gnr_name':'song_gnr_lst'}, inplace=True)


In [25]:
song_meta_db.head()

,song_id,song_name,artist_name_lst,song_gnr_lst
0,0,Feelings,[Various Artists],[POP]
1,1,"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],[클래식]
2,2,Solsbury Hill (Remastered 2002),[Peter Gabriel],[POP]
3,3,Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],[일렉트로니카]
4,4,그남자 그여자,[Jude Law],[뉴에이지]


In [26]:
# db_song_meta 저장
song_meta_db.to_json('../0_data/db_song_meta.json', orient='records')

### 2. playlists

> 0. 장르정보, GN9000대분류 장르였던 노래들 삭제

In [27]:
# song_meta에서 삭제한 to_del_song_id들을 playlists에서도 삭제
playlists['songs'] = playlists['songs'].apply(lambda x: [song for song in x if song not in to_del_song_id])

In [28]:
# songs가 비어 있는 행이 있는지 확인 -> 10개(아기 클래식, 뮤지컬 동화, 자장가 등)
playlists[playlists['songs'].apply(lambda x: len(x) == 0)] 

# 비어 있는 행은 삭제
playlists = playlists[playlists['songs'].apply(lambda x: len(x) != 0)] 

In [29]:
playlists.info() # 10행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115061 entries, 0 to 115070
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          115061 non-null  object
 1   id            115061 non-null  int64 
 2   plylst_title  115061 non-null  object
 3   songs         115061 non-null  object
 4   like_cnt      115061 non-null  int64 
 5   updt_date     115061 non-null  object
dtypes: int64(2), object(4)
memory usage: 6.1+ MB


> 1. 타이틀, 태그 불용어 처리

- 타이틀 특수문자 제거

In [30]:
# 이모지 제거 함수
def replace_emoji(inputString):
    return core.replace_emoji(inputString, replace='')

# 특수문자 제거 함수
def clean_text(inputString):
  text_rmv = re.sub('[-=+,#/\?:^●○.@*\"※~ㆍ!<>$♥❤♡☆★♪♬♩¶♨①๑｡＊②ఇ¥◈ ͡✔ ͡° ͜ʖ▶◀◎ღ✿ﾉ ;\\\_■『│』ᕷⓗ‘|\(\)\[\]`\'…》\”\“\’·]',' ', inputString)
  return text_rmv

In [31]:
# # 이모지 제거 함수 적용
playlists['plylst_title'] = playlists['plylst_title'].apply(replace_emoji)

# 특수문자 제거 함수 적용
playlists['plylst_title'] = playlists['plylst_title'].apply(clean_text)

In [32]:
# 양 옆 공백, %, & 제거

playlists['plylst_title'] = playlists['plylst_title'].str.strip()
playlists['plylst_title'] = playlists['plylst_title'].str.strip('%')
playlists['plylst_title'] = playlists['plylst_title'].str.strip('&')

In [33]:
playlists[playlists['id'].isin([22525, 143039])]

,tags,id,plylst_title,songs,like_cnt,updt_date
56134,[흥폭발],22525,내적댄스 터져베이베 아이돌 노래노래,"[108003, 594076, 297411, 97189, 35713, 445680,...",41,2018-06-15 19:24:26.000
59162,"[가을, 설렘, 사랑]",143039,가을이다 o 곧 겨울이다 T T,"[13142, 5049, 503816, 70166, 578529, 336829, 2...",1,2018-07-31 10:55:04.000


- 태그 특수문자 제거

In [34]:
# 특수문자 '_'가 있는 행 찾기
# '_'가 있는 행 찾기
rows_with_underscore = playlists['tags'][playlists['tags'].apply(lambda x: any('_' in tag for tag in x))]
rows_with_underscore

148                            [비오는날, 비_오는_날, 휴식, 비_오는날, 비]
285                                [상쾌해지는, 호불호없는, 모두가_흥얼흥얼]
364        [추천, 일렉트로니카, 5월_4주차, 주간, 최신, 전지음악, 인기, 주간일렉트로니카]
409                      [들었으면, 인디음악, 한번쯤, 이미지는_직접찍은사진, 인디]
522       [힙합, label_crew, 랩, 국내힙합, 트렌드, 쇼미더머니, 스웨그, aom...
                                ...                        
114625                                   [칠_Chill, 설레는_달콤함]
114647                     [우울, 위로, 위로가_필요할_때, 슬픔, 이별, 힘든날]
114688    [어쿠스틱, 혼자, 발라드, 우울, 분위기, 멜로디, 띵곡들, 조용한, 자기_전, 인디]
114703    [패션위크, 뉴욕, 도시, 갈라쇼음악, 트렌디, 팝, 편집샵, i_know_it, ...
115068               [담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]
Name: tags, Length: 2252, dtype: object

In [35]:
# 특수 문자를 제거하는 함수 정의
def remove_special_characters(tag_list):
    cleaned_tags = []
    for tag in tag_list:
        cleaned_tag = re.sub(r'[^\uAC00-\uD7A30-9a-zA-Z\s]', '', tag).replace(" ", "")
        cleaned_tags.append(cleaned_tag)
    return cleaned_tags

# 함수 적용
playlists['tags'] = playlists['tags'].apply(remove_special_characters)

In [36]:
playlists[playlists['id'].isin([32533, 151357, 11343])]

,tags,id,plylst_title,songs,like_cnt,updt_date
148,"[비오는날, 비오는날, 휴식, 비오는날, 비]",32533,비오는날 감성 자극 듣기 좋은 어쿠스틱 피아노 발라드,"[133091, 527764, 468136, 260894, 22845, 148876...",14,2020-02-16 00:56:45.000
285,"[상쾌해지는, 호불호없는, 모두가흥얼흥얼]",151357,기분좋아지는 노래들,"[270061, 158766, 573179, 272062, 37027, 245028...",0,2019-11-24 05:49:55.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의이야기, 사랑, 친구]",11343,1 눈물이 앞을 가리는 나의 이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000


In [37]:
# 플레이리스트 타이틀이 1글자인 조건 만들기
playlists_title_1str = playlists[playlists['plylst_title'].str.len() == 1]
playlists_title_1str = list(playlists_title_1str['id'].values) # 24 -> 롹, 잠, 쿨, 힙 등이며 대부분 좋아요 0 또는 1
# playlists[playlists['id'] == 98628]

# 삭제
playlists = playlists[playlists['plylst_title'].str.len() != 1]

In [38]:
# 빈 타이틀 삭제
playlists = playlists[playlists['plylst_title'] != '']

# 숫자로만 구성된 타이틀 삭제
numeric_title_rows = playlists[playlists['plylst_title'].str.isnumeric()]
indices_to_remove = numeric_title_rows.index
playlists = playlists.drop(indices_to_remove)

playlists.info()


# 115061  - 114697  = 364행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114697 entries, 0 to 115070
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          114697 non-null  object
 1   id            114697 non-null  int64 
 2   plylst_title  114697 non-null  object
 3   songs         114697 non-null  object
 4   like_cnt      114697 non-null  int64 
 5   updt_date     114697 non-null  object
dtypes: int64(2), object(4)
memory usage: 6.1+ MB


> 2. 타이틀 중복 제거(기준 : 노래 개수 or 좋아요 개수)

In [39]:
# 1. 플레이리스트 아이디(id)와 플레이리스트 명(plylst_title) 추출
plylst_title = playlists[['id', 'plylst_title']]

# 2. 플레이리스트 이름 별 플레이리스트 ID count 테이블 생성 : plylst_id_cnt
plylst_id_cnt = plylst_title.groupby('plylst_title').id.nunique().reset_index(name = 'mapping_plylst_cnt')

# 3. 플레이리스트 명 중복 구분 : 서로 다른 플레이리스트 아이디로 겹치는 플레이리스트 이름이 없으면 '중복 없음', 그 외 '중복 있음'
plylst_id_cnt = plylst_id_cnt.assign(
    mapping_plylst_cnt_category = pd.cut(plylst_id_cnt['mapping_plylst_cnt'], [0, 1, np.inf], labels = ['중복 없음', '중복 있음'])
)
plylst_id_cnt
# [0, 1, np.inf] : 0이상 1미만 값 = 중복없음, 1이상의 값 = 중복있음

# 4. 중복 구분 별 플레이리스트 수 count 테이블 생성 : plylst_id_cnt_division
plylst_id_cnt_division = pd.DataFrame(plylst_id_cnt.groupby('mapping_plylst_cnt_category').count())
plylst_id_cnt_division

,plylst_title,mapping_plylst_cnt
mapping_plylst_cnt_category,,
중복 없음,106515,106515
중복 있음,2722,2722


In [40]:
# 플레이리스트 타이틀을 기준으로 merge해 중복된 플레이리스트만 확인
playlists_dup = pd.merge(playlists, plylst_id_cnt, on='plylst_title')
playlists_dup= playlists_dup[playlists_dup['mapping_plylst_cnt_category'] == '중복 있음']
playlists_dup['song_cnt'] = playlists_dup['songs'].apply(len)
playlists_dup['tag_cnt'] = playlists_dup['tags'].apply(len)
playlists_dup.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
16,"[감성, 질리지않는, 나만알고싶은, Pop]",1516,나만 알고싶은 노래들,"[331055, 99287, 310974, 376435, 146989, 430106...",1,2018-01-02 12:10:59.000,4,중복 있음,31,4
17,"[기분전환, 까페]",86179,나만 알고싶은 노래들,"[480142, 578916, 467225, 701072, 197046, 63557...",6,2016-10-06 02:08:40.000,4,중복 있음,46,2
18,"[숨은명곡, 좋은노래]",54673,나만 알고싶은 노래들,"[113618, 422482, 380069, 75425, 79141, 657137,...",7,2016-07-15 10:20:17.000,4,중복 있음,23,2
19,"[감성힙합, 나만알고싶은, 뿜뿜, 그루브, 노래, 감성]",135616,나만 알고싶은 노래들,"[470548, 603591, 382764, 288501, 187291, 37802...",30,2018-07-13 08:18:54.000,4,중복 있음,45,6
44,"[기분전환, 가을]",105328,트렌디한 Cafe Pop Music,"[419702, 403579, 472776, 502078, 662300, 44363...",55,2014-09-18 11:29:55.000,2,중복 있음,16,2


In [41]:
# 같은 타이틀을 가진 플레이리스트의 노래 개수 비교
# playlists_dup에서 타이틀별로 groupby 후 song_cnt 값들을 song_cnt_nunique(새 컬럼)에 리스트 형태로 넣은 후 중복을 없애기 위해 set 진행
# 1의 의미는 노래 갯수가 중복되어 있다는 뜻이기 때문에 1의 갯수가 적은 지표를 기준으로 중복 플레이리스트 제거를 진행해야함
from collections import Counter

playlists_dup_song_cnt = playlists_dup.groupby('plylst_title')['song_cnt'].apply(list).reset_index()
playlists_dup_song_cnt['song_cnt_nunique'] = playlists_dup_song_cnt['song_cnt'].apply(lambda x: len(set(x)))
Counter(playlists_dup_song_cnt['song_cnt_nunique'])

Counter({2: 1544,
         1: 507,
         3: 344,
         4: 131,
         5: 58,
         6: 45,
         7: 30,
         9: 6,
         14: 5,
         20: 2,
         8: 20,
         15: 5,
         11: 7,
         10: 6,
         16: 3,
         13: 3,
         25: 1,
         12: 2,
         19: 1,
         22: 1,
         17: 1})

In [42]:
# 같은 타이틀을 가진 플레이리스트의 태그 개수 비교

playlists_dup_tag_cnt = playlists_dup.groupby('plylst_title')['tag_cnt'].apply(list).reset_index()
playlists_dup_tag_cnt['tag_cnt_nunique'] = playlists_dup_tag_cnt['tag_cnt'].apply(lambda x: len(set(x)))
Counter(playlists_dup_tag_cnt['tag_cnt_nunique'])

Counter({1: 1225, 3: 183, 2: 1222, 4: 59, 5: 20, 6: 9, 7: 4})

In [43]:
# 같은 타이틀을 가진 플레이리스트의 좋아요 비교

playlists_dup_like_cnt = playlists_dup.groupby('plylst_title')['like_cnt'].apply(list).reset_index()
playlists_dup_like_cnt['like_cnt_nunique'] = playlists_dup_like_cnt['like_cnt'].apply(lambda x: len(set(x)))
Counter(playlists_dup_like_cnt['like_cnt_nunique'])

Counter({1: 431,
         2: 1741,
         3: 311,
         4: 102,
         5: 54,
         6: 32,
         7: 14,
         11: 4,
         9: 7,
         13: 2,
         10: 4,
         12: 5,
         8: 10,
         15: 1,
         14: 2,
         18: 1,
         24: 1})

- 플레이리스트 타이틀이 중복인 플레이리스트만 따로 모으고
- 그 중복 플리모음 중 노래갯수가 가장 많은 하나만 남겨두고
- 원래 train의 중복 없음만 따로 모아 중복모음과 concat

In [44]:
# train에 중복 갯수, 중복 여부 컬럼 추가, 플레이리스트 내 곡 갯수 추가
playlists_add_col = pd.merge(playlists, plylst_id_cnt, on='plylst_title')
playlists_add_col['song_cnt'] = playlists_add_col['songs'].apply(lambda x : len(x))

# 매핑갯수, 타이틀, 플리 내 노래 갯수, like_cnt 내림차순 정렬
playlists_filter = playlists_add_col.sort_values(['mapping_plylst_cnt', 'plylst_title', 'song_cnt', 'like_cnt'], ascending=False)
playlists_filter

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt
8531,"[집중, 어쿠스틱, 혼자, 발라드, 우울, 드라이브, 산책, 조용조용, 띵곡들, 인디]",152785,혼자만의 시간이 필요할때,"[208812, 629738, 232874, 13281, 193232, 389468...",1,2020-03-26 06:14:54.000,43,중복 있음,81
8518,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 띵곡만, 산책, 운전, 인디]",127180,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-03-24 06:39:20.000,43,중복 있음,78
8535,"[띵곡, 혼자만, 어쿠스틱, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 인디]",76448,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-04-13 05:56:55.000,43,중복 있음,78
8540,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 띵곡들, 인디]",141114,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-22 06:38:57.000,43,중복 있음,78
8547,"[어쿠스틱, 혼자, 발라드, 분위기, 드라이브, 조용한, 산책, 운전, 띵곡들, 인디]",18201,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-06 05:20:46.000,43,중복 있음,78
...,...,...,...,...,...,...,...,...,...
31841,[일렉],56144,00 Party 일렉트로댄스,"[256418, 183424, 431998, 654442, 354592, 24132...",7,2018-02-06 16:02:42.000,1,중복 없음,20
17803,[일렉],58768,00 Electronica EDM,"[673003, 251403, 617690, 441958, 436128, 62788...",5,2020-01-05 00:24:22.000,1,중복 없음,133
108575,[일렉],110536,0 일렉듣고 광질준비 O K ElecVVIP,"[349306, 93890, 651902, 46845, 362347, 330768,...",173,2012-11-29 14:34:56.000,1,중복 없음,59
52777,"[비오는날, 추억, 회상]",106660,비오는날 듣기 좋은 연주곡 모음,"[79329, 51922, 173875, 216115, 422584, 509605,...",6,2015-10-06 11:45:24.000,1,중복 없음,25


In [45]:
playlists_dup = playlists_filter[playlists_filter['mapping_plylst_cnt_category'] == '중복 있음'] # 중복 플리 모음
playlists_dup['plylst_title'].nunique() # 중복된 플레이리스트의 유니크 타이틀 갯수 : 2722

2722

In [46]:
playlists_dup['tag_cnt'] = playlists_dup['tags'].apply(len)
playlists_dup.head()

C:\Users\cg063\AppData\Local\Temp\ipykernel_12508\2448186990.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlists_dup['tag_cnt'] = playlists_dup['tags'].apply(len)


,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
8531,"[집중, 어쿠스틱, 혼자, 발라드, 우울, 드라이브, 산책, 조용조용, 띵곡들, 인디]",152785,혼자만의 시간이 필요할때,"[208812, 629738, 232874, 13281, 193232, 389468...",1,2020-03-26 06:14:54.000,43,중복 있음,81,10
8518,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 띵곡만, 산책, 운전, 인디]",127180,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-03-24 06:39:20.000,43,중복 있음,78,10
8535,"[띵곡, 혼자만, 어쿠스틱, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 인디]",76448,혼자만의 시간이 필요할때,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-04-13 05:56:55.000,43,중복 있음,78,10
8540,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 조용한, 산책, 띵곡들, 인디]",141114,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-22 06:38:57.000,43,중복 있음,78,10
8547,"[어쿠스틱, 혼자, 발라드, 분위기, 드라이브, 조용한, 산책, 운전, 띵곡들, 인디]",18201,혼자만의 시간이 필요할때,"[484964, 626369, 570836, 366786, 413837, 20913...",1,2020-04-06 05:20:46.000,43,중복 있음,78,10


In [47]:
# playlists_dup 중 각 플리 제목 별 노래갯수가 가장 많은 하나만 남긴다
playlists_dup_filter = pd.DataFrame(playlists_dup[playlists_dup.groupby('plylst_title')['song_cnt'].transform(max) == playlists_dup['song_cnt']]) # 3572 
playlists_dup_filter = pd.DataFrame(playlists_dup_filter[playlists_dup_filter.groupby('plylst_title')['like_cnt'].transform(max) == playlists_dup_filter['like_cnt']]) # 2928 
playlists_dup_filter = pd.DataFrame(playlists_dup_filter[playlists_dup_filter.groupby('plylst_title')['tag_cnt'].transform(max) == playlists_dup_filter['tag_cnt']]) # 2909 
playlists_dup_filter = pd.DataFrame(playlists_dup_filter[playlists_dup_filter.groupby('plylst_title')['updt_date'].transform(max) == playlists_dup_filter['updt_date']]) # 2748 
playlists_dup_filter

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
8531,"[집중, 어쿠스틱, 혼자, 발라드, 우울, 드라이브, 산책, 조용조용, 띵곡들, 인디]",152785,혼자만의 시간이 필요할때,"[208812, 629738, 232874, 13281, 193232, 389468...",1,2020-03-26 06:14:54.000,43,중복 있음,81,10
3119,[비오는날듣기좋은노래],75012,비오는날,"[65672, 522825, 145252, 146780, 500731, 473608...",22,2017-08-19 15:16:36.000,34,중복 있음,200,1
5536,"[조용히, 어쿠스틱, 여유, 발라드, 분위기, 힐링, 자기전이나, 띵곡, 인디, 편안한]",52312,자기전 하루한곡 발라드,"[570836, 209135, 389920, 105140, 582252, 38261...",1,2020-04-02 06:54:25.000,31,중복 있음,82,10
8428,[금요일],48129,금요일,"[614756, 147427, 340332, 675466, 414036, 61619...",0,2017-03-10 17:31:34.000,30,중복 있음,20,1
7287,[발라드],5960,추억,"[208821, 481749, 565402, 362197, 407236, 46604...",10,2010-10-21 09:45:12.000,27,중복 있음,85,1
...,...,...,...,...,...,...,...,...,...,...
155,"[기분좋은, 100번]",148495,100번 들어도 기분좋은 가요,"[208186, 155952, 669617, 438439, 397412, 60398...",4,2016-10-31 13:59:41.000,2,중복 있음,16,2
76630,[Pop],62278,100% 로맨틱 조지 마이클 223,"[366485, 435804, 442680, 538949, 263360, 29495...",0,2019-03-27 15:27:44.000,2,중복 있음,34,1
34439,[팝],50564,1 HITS OF DECADE SPECIAL 2,"[60025, 326802, 82991, 533287, 404318, 82686, ...",5,2009-07-11 21:37:42.000,2,중복 있음,37,1
75844,[팝],109558,1 HITS OF DECADE SPECIAL,"[289618, 78629, 349460, 218356, 189583, 613209...",5,2009-07-05 18:23:47.000,2,중복 있음,39,1


In [48]:
playlists_dup_filter[playlists_dup_filter['plylst_title'] == 'qqqa'] # 필터링 되지 않은 중복 타이틀은 플레이리스트 자체가 같음

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category,song_cnt,tag_cnt
50436,[여름],7524,qqqa,"[522825, 455668, 549178, 407828, 343974]",1,2015-11-04 17:51:14.000,3,중복 있음,5,1
50437,[여름],43031,qqqa,"[522825, 455668, 549178, 407828, 343974]",1,2015-11-04 17:51:14.000,3,중복 있음,5,1


In [49]:
# 타이틀이 같은 플레이리스트 중 하나만 남긴다
playlists_dup_filter = pd.DataFrame(playlists_dup_filter.groupby('plylst_title', group_keys=False).apply(lambda x: x.loc[x['song_cnt'].idxmax()]))
playlists_dup_filter = playlists_dup_filter.reset_index(drop=True) # 2722

In [50]:
# playlists_dup_filter 중 중복 여부 확인

# 1. 플레이리스트 아이디(id)와 플레이리스트 명(plylst_title) 추출
plylst_title_2 = playlists_dup_filter[['id', 'plylst_title']]

# 2. 플레이리스트 이름 별 플레이리스트 ID count 테이블 생성 : plylst_id_cnt
plylst_id_cnt_2 = plylst_title_2.groupby('plylst_title').id.nunique().reset_index(name = 'mapping_plylst_cnt')

# 3. 플레이리스트 명 중복 구분 : 서로 다른 플레이리스트 아이디로 겹치는 플레이리스트 이름이 없으면 '중복 없음', 그 외 '중복 있음'
plylst_id_cnt_2 = plylst_id_cnt_2.assign(
    mapping_plylst_cnt_category = pd.cut(plylst_id_cnt_2['mapping_plylst_cnt'], [0, 1, np.inf], labels = ['중복 없음', '중복 있음'])
)
plylst_id_cnt_2
# [0, 1, np.inf] : 0이상 1미만 값 = 중복없음, 1이상의 값 = 중복있음

# 4. 중복 구분 별 플레이리스트 수 count 테이블 생성 : plylst_id_cnt_division
plylst_id_cnt_division_2 = pd.DataFrame(plylst_id_cnt_2.groupby('mapping_plylst_cnt_category').count())
plylst_id_cnt_division_2

,plylst_title,mapping_plylst_cnt
mapping_plylst_cnt_category,,
중복 없음,2722,2722
중복 있음,0,0


In [51]:
# 중복 없음 파일만 만들기
playlists_no_dup = pd.merge(playlists, plylst_id_cnt, on='plylst_title')
playlists_no_dup = playlists_no_dup[playlists_no_dup['mapping_plylst_cnt_category'] == '중복 없음']
playlists_no_dup # 106515행

,tags,id,plylst_title,songs,like_cnt,updt_date,mapping_plylst_cnt,mapping_plylst_cnt_category
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,중복 없음
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,1,중복 없음
2,"[까페, 잔잔한]",76951,편하게 잔잔하게 들을 수 있는 곡,"[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,1,중복 없음
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,1,중복 없음
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,1,중복 없음
...,...,...,...,...,...,...,...,...
114692,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E SM 2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,1,중복 없음
114693,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,1,중복 없음
114694,"[담시, 가족, 눈물, 그리움, 주인공, 나의이야기, 사랑, 친구]",11343,1 눈물이 앞을 가리는 나의 이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,1,중복 없음
114695,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,1,중복 없음


In [52]:
playlists_dup_proc = pd.concat([playlists_dup_filter, playlists_no_dup], axis=0)
playlists_dup_proc = playlists_dup_proc.iloc[:, :6]
playlists_dup_proc.info()

# 2722+106515 =109237
# 114697 - 109237 = 타이틀 중복 제거로 5460행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109237 entries, 0 to 114696
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   tags          109237 non-null  object
 1   id            109237 non-null  int64 
 2   plylst_title  109237 non-null  object
 3   songs         109237 non-null  object
 4   like_cnt      109237 non-null  int64 
 5   updt_date     109237 non-null  object
dtypes: int64(2), object(4)
memory usage: 5.8+ MB


> 3. 플레이리스트 별 노래 개수, 좋아요 개수 이상치 행 날리기

- 노래 개수 5~104개 가진 플레이리스트만 남기기

In [53]:
playlists_dup_proc['songs_cnt'] = playlists_dup_proc['songs'].apply(len)
playlists_dup_proc['songs_cnt'].describe()

count    109237.000000
mean         45.949321
std          44.216494
min           1.000000
25%          19.000000
50%          29.000000
75%          53.000000
max         200.000000
Name: songs_cnt, dtype: float64

In [54]:
Q1 = 19
Q3 = 53
IQR = Q3 - Q1

print(Q1-(1.5*IQR))
print(Q3+(1.5*IQR))

-32.0
104.0


In [55]:
playlists_dup_proc['songs_cnt'] = playlists_dup_proc['songs'].apply(lambda x : len(x))
playlists_song_cut = playlists_dup_proc[(playlists_dup_proc['songs_cnt'] >= 5) & (playlists_dup_proc['songs_cnt'] < 105) ]  # 5~104개만 남기기
playlists_song_cut.info() # 109237-98973 = 10264행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98973 entries, 0 to 114695
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          98973 non-null  object
 1   id            98973 non-null  int64 
 2   plylst_title  98973 non-null  object
 3   songs         98973 non-null  object
 4   like_cnt      98973 non-null  int64 
 5   updt_date     98973 non-null  object
 6   songs_cnt     98973 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 6.0+ MB


- 좋아요 2개 이하, 6개 이하(describe 시 25%가 2개임, 다시 describe했을 때 25% = 6) 삭제

In [56]:
playlists_song_cut['like_cnt'].describe()

count    98973.000000
mean        77.190284
std        437.540113
min          0.000000
25%          2.000000
50%          7.000000
75%         22.000000
max      53211.000000
Name: like_cnt, dtype: float64

In [57]:
Q1 = 2
Q3 = 22
IQR = Q3 - Q1

print(Q1-(1.5*IQR))
print(Q3+(1.5*IQR))

-28.0
52.0


In [58]:
playlists_song_like_cut = playlists_song_cut[playlists_song_cut['like_cnt'] > 2]

In [59]:
playlists_song_like_cut['like_cnt'].describe()

count    74114.000000
mean       102.757657
std        503.042558
min          3.000000
25%          6.000000
50%         12.000000
75%         35.000000
max      53211.000000
Name: like_cnt, dtype: float64

In [60]:
playlists_song_like_cut = playlists_song_cut[playlists_song_cut['like_cnt'] > 6]
playlists_song_like_cut = playlists_song_like_cut.iloc[:, :6]
playlists_song_like_cut.info()# 98973 - 51916 = 47057행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51916 entries, 8 to 114693
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          51916 non-null  object
 1   id            51916 non-null  int64 
 2   plylst_title  51916 non-null  object
 3   songs         51916 non-null  object
 4   like_cnt      51916 non-null  int64 
 5   updt_date     51916 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.8+ MB


> 4. song, 태그 빈도수에 따라 리스트에서 빼기 -> 리스트가 비워져있으면 행 날리기

- 10회 이하 사용된 tag 리스트에서 빼기

In [63]:
pd.DataFrame(playlists_song_cut.explode('tags')['tags']).nunique() # 26837개

tags    26837
dtype: int64

In [64]:
# tag usage count 계산
tag_counts = playlists_song_like_cut.explode('tags')['tags'].value_counts(ascending=True)

# 10번 사용된 태그
less_than_10_tags = list(tag_counts[tag_counts <= 10].index) # 17935개

In [65]:
len(less_than_10_tags)

17935

- tag_counts describe

In [66]:
tag_counts.describe()

count    19601.000000
mean        12.124024
std        134.788890
min          1.000000
25%          1.000000
50%          1.000000
75%          3.000000
max       7303.000000
Name: tags, dtype: float64

In [67]:
Q1 = 1
Q3 = 3
IQR = Q3 - Q1

print(Q1-(1.5*IQR))
print(Q3+(1.5*IQR))

-2.0
6.0


In [68]:
from tqdm import tqdm
tqdm.pandas()

playlists_song_like_cut['tags'] = playlists_song_like_cut['tags'].progress_apply(lambda tags: list(set(tags).difference(less_than_10_tags)))

  0%|          | 0/51916 [00:00<?, ?it/s]

100%|██████████| 51916/51916 [00:38<00:00, 1335.27it/s]


In [69]:
# tags가 비어있지 않은 행만 남김
playlists_tag_cut = playlists_song_like_cut[playlists_song_like_cut['tags'].apply(lambda x: len(x) > 0)]
playlists_tag_cut.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
8,"[크리스마스, 겨울]",117162,12월의 재즈,"[588425, 407630, 235568, 347651, 423132, 35204...",353,2016-12-26 12:33:33.000
9,[발라드],151638,12월의 크리스마스를 yeah,"[290351, 404382, 591246, 18938, 610105, 676958...",29,2009-12-22 09:11:45.000
12,[일렉],27984,1월의 최신 EDM 소식,"[329736, 201677, 626680, 555898, 58608, 585855...",84,2015-01-29 16:22:19.000
18,"[랩, 힙합]",48636,2000년대 힙합을 부탁해,"[690359, 316577, 492528, 320162, 362619, 31150...",54,2015-06-09 17:25:51.000
21,[락],30079,2008 그랜드민트 페스티벌,"[4687, 447505, 464502, 409059, 24344, 564377, ...",12,2008-12-23 19:56:46.000


In [70]:
playlists_tag_cut['tag_cnt'] = playlists_tag_cut['tags'].apply(len)
playlists_tag_cut['tag_cnt'].unique()

C:\Users\cg063\AppData\Local\Temp\ipykernel_12508\4293636261.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlists_tag_cut['tag_cnt'] = playlists_tag_cut['tags'].apply(len)


array([ 2,  1,  4,  3,  7,  8,  6,  5,  9, 10, 11], dtype=int64)

In [71]:
playlists_song_like_cut.explode('tags')['tags'].value_counts(ascending=True)

뉴웨이브       11
SOCAR      11
쏘카         11
해외알앤비      11
편한         11
         ... 
잔잔한      4294
휴식       4703
감성       4827
드라이브     4839
기분전환     7302
Name: tags, Length: 1666, dtype: int64

In [72]:
playlists_tag_cut = playlists_tag_cut.iloc[:, :6]
playlists_tag_cut.info()

# 51916 - 50984  = 932행 삭제

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50984 entries, 8 to 114693
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          50984 non-null  object
 1   id            50984 non-null  int64 
 2   plylst_title  50984 non-null  object
 3   songs         50984 non-null  object
 4   like_cnt      50984 non-null  int64 
 5   updt_date     50984 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.7+ MB


In [73]:
pd.DataFrame(playlists_tag_cut.explode('tags')['tags']).nunique() # 26837개 -> 1666개

tags    1666
dtype: int64

- 10회 이하 사용된 song 리스트에서 빼기

In [74]:
# song usage count 계산
songs_counts = playlists_tag_cut.explode('songs')['songs'].value_counts(ascending=True)

# 10번 이하 사용된 song
less_than_10_songs = list(songs_counts[songs_counts <= 10].index) # 358521개

In [75]:
pd.DataFrame(playlists_tag_cut.explode('songs')['songs']).nunique() # 390852개

songs    390852
dtype: int64

- songs_counts describe 확인

In [76]:
songs_counts.describe()

count    390852.000000
mean          4.912806
std          15.773784
min           1.000000
25%           1.000000
50%           1.000000
75%           3.000000
max         893.000000
Name: songs, dtype: float64

In [77]:
Q1 = 1
Q3 = 3
IQR = Q3 - Q1

print(Q1-(1.5*IQR))
print(Q3+(1.5*IQR))

-2.0
6.0


In [78]:
# 10번 이하 사용된 songs 삭제
playlists_tag_cut['songs'] = playlists_tag_cut['songs'].progress_apply(lambda songs: list(set(songs).difference(less_than_10_songs)))

100%|██████████| 50984/50984 [09:29<00:00, 89.48it/s] 


In [79]:
# songs가 비어있지 않은 행만 남김
playlists_tag_song_cut = playlists_tag_cut[playlists_tag_cut['songs'].apply(lambda x: len(x) > 0)]
playlists_tag_song_cut.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
8,"[크리스마스, 겨울]",117162,12월의 재즈,"[677632, 309377, 473793, 179652, 407630, 35204...",353,2016-12-26 12:33:33.000
9,[발라드],151638,12월의 크리스마스를 yeah,"[560007, 653711, 112399, 404382, 528414, 65475...",29,2009-12-22 09:11:45.000
12,[일렉],27984,1월의 최신 EDM 소식,"[16261, 596152]",84,2015-01-29 16:22:19.000
18,"[랩, 힙합]",48636,2000년대 힙합을 부탁해,"[168900, 514885, 399897, 310879, 316577, 32016...",54,2015-06-09 17:25:51.000
21,[락],30079,2008 그랜드민트 페스티벌,"[186245, 581896, 310677, 564377, 132247, 69155...",12,2008-12-23 19:56:46.000


In [84]:
playlists_tag_song_cut['song_cnt'] = playlists_tag_song_cut['songs'].apply(len)
playlists_tag_song_cut['song_cnt'].unique()

C:\Users\cg063\AppData\Local\Temp\ipykernel_12508\1943819515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlists_tag_song_cut['song_cnt'] = playlists_tag_song_cut['songs'].apply(len)


array([  8,  27,   2,  14,  23,  58,  19,   1,   5,   4,  26,   7,  12,
         9,  11,  24,  59,  10,  60,  16,  64,  30,  66,  44,  13,  76,
        90,  71,  21,  31,  68,  88,   6,  28,  22,  18,  39,  40,  20,
        49,  36,  29,  53,  25,  17,  52,  56,  57,  81,  41,   3,  33,
        50,  46,  79,  83,  42,  65,  98,  37,  43,  47,  51,  15,  74,
        32,  35,  84,  34,  38,  80,  89,  70,  61,  63,  72,  55,  73,
        54,  62,  82,  77,  45,  78,  48,  87,  93,  97,  67,  75,  95,
        99, 101, 100,  91,  86,  94,  92,  69,  85, 103,  96, 102, 104],
      dtype=int64)

In [85]:
playlists_tag_song_cut.explode('songs')['songs'].value_counts(ascending=True) # 10회 이하 사용된 song들이 알맞게 제거 되었는지 확인

73450      11
701433     11
103239     11
433799     11
369510     11
         ... 
581799    583
144663    635
366786    678
116573    743
205179    893
Name: songs, Length: 32331, dtype: int64

In [89]:
pd.DataFrame(playlists_tag_song_cut.explode('songs')['songs']).nunique() # 390852개 -> 32331개로 삭제
# 사용빈도수가 높은 노래가 좋아요를 유도한다는 근거를 만듦 <- 회귀분석을 통해서
# 노래들간의 관계가 차원축소되고 이걸 오토인코더에넣으면 노래들간의 관계를 복원한다는 아이디어
# 노래들간의 관계를 똑같이 복원하면 안되니까 디노이즈

songs    32331
dtype: int64

In [87]:
playlists_tag_song_cut = playlists_tag_song_cut.iloc[:, :6]
playlists_tag_song_cut.info()

# 50984 - 47989 = 2995행 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47989 entries, 8 to 114693
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          47989 non-null  object
 1   id            47989 non-null  int64 
 2   plylst_title  47989 non-null  object
 3   songs         47989 non-null  object
 4   like_cnt      47989 non-null  int64 
 5   updt_date     47989 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.6+ MB


In [88]:
# playlists 저장
playlists_tag_song_cut.to_json('../0_data/playlists_tag,song_10freq.json', orient='records')

> 정리

=> song_meta : 날짜 수정, 장르 정보 없는 노래 제거  
=> playlists  
    - 타이틀 & 태그 불용어 삭제 : 특수문자, 빈 타이틀, 모두 숫자인 경우, 1글자인 경우 포함  
    - 타이틀 중복 제거 : 타이틀이 완벽하게 같을 경우 '1_노래 갯수 많은 순서 -> 2_좋아요 갯수 많은 순서 -> 3_태그 갯수 많은 순서 -> 4_최신 업데이트' 기준으로 1개씩 골라냄 (4가지를 했는데도 같은건 완벽히 같았기에 하나만 남김)  
    - 중복 제거 후 song_cnt describe 해 5~104개 남기기, like_cnt describe 3개 이상인 플레이리스트   
    - 1번 사용된 tag, 2번 이하 사용된song 제거  
  
  
=> 115071(원래 playlist) -73014(전처리 후) = 42057행 삭제

> 데이터 저장

- 최종 전처리 파일은 ../0_data/playlists_filter.json 으로 저장

In [368]:
# playlists 저장
playlists_tag_song_cut.to_json('../0_data/playlists_filter.json', orient='records')

---